<img src="images/ProjectPythia_Logo_Final-01-Blue.svg" width=250 alt="Project Pythia Logo"></img>
<img src="https://intake.readthedocs.io/en/latest/_static/images/logo.png" width=250 alt="Intake Logo"></img>
<img src="https://cmip6-preprocessing.readthedocs.io/en/latest/_images/logo.png" width=250 alt="CMIP6 Preprocessing Logo"></img>

# Intake-ESM and CMIP6 Preprocessing

## Overview

1. Use Intake-ESM to browse and load CMIP6 data
1. Use CMIP6 Preprocessing to make analysis easier
    1. Rename coordinates
    1. Load and merge cell area data
    1. Unify time across models (currently not working)

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |

- **Time to learn**: 30 minutes

## Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import xarrayutils as xru
xr.set_options(display_style='html')
import intake
import dask

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = 12, 6

## Using Intake-ESM to load CMIP6 data

Intake-ESM is a package that aims to provide a higher-level interface to searching and loading Earth System Model data archives, such as CMIP. Here we are opening the `pangeo-cmip6.json` catalog of CMIP6 Zarr data that is stored in Pangeo's Google Storage.

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

Now we can search the catalog with `.search()`. For example, this returns the monthly average (Omon) downward heat flux at the ocean surface (hfds) for 1%/year CO2 ramp experiments (1pctCO2) on the model's native grid (gn) for models from a couple institutions, exported as a Pandas dataframe (`.df`).

In [ ]:
cat = col.search(institution_id=['NCAR', 'MPI-M'], experiment_id='1pctCO2', member_id='r1i1p1f1', table_id='Omon', variable_id='hfds', grid_label='gn')
cat.df

In [ ]:
cat_test = col.search(experiment_id='1pctCO2', table_id='Omon', variable_id='hfds', grid_label='gn')
cat_test.df

Intake knows how to automatically open the datasets using Xarray, including the ability to merge them into a dictionary of datasets.

In [ ]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True})
list(dset_dict.keys())

We can then access one of the datasets with one of the keys listed above.

In [ ]:
cesm2 = dset_dict['CMIP.NCAR.CESM2.1pctCO2.Omon.gn']
cesm2

In [ ]:
cesm2.hfds.isel(time=0).plot()

## CMIP6 Preprocessing

### Combined preprocessing

While this provides easy access to the individual models, one problem with trying to do inter-model analysis is that the models are mostly on different grids and sometimes have inconsistent dimension and coordinate names. Here we will use the `combined_preprocessing` option from the `cmip6_preprocessing` package. For more details, see the [tutorial](https://cmip6-preprocessing.readthedocs.io/en/latest/tutorial.html) on the [CMIP6 Preprocessing website](https://cmip6-preprocessing.readthedocs.io/en/latest/index.html).

In [ ]:
from cmip6_preprocessing.preprocessing import combined_preprocessing, replace_x_y_nominal_lat_lon

Let's look at the same data as before, but with the additional `preprocess` argument.

In [ ]:
with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    preproc_dset_dict = cat.to_dataset_dict(zarr_kwargs={'consolidated': True, 'decode_times': True, 'use_cftime': True}, aggregate=False, preprocess=combined_preprocessing)

Let's look at a couple Datasets to see what this preprocessing changed.

In [ ]:
dset_dict['CMIP.NCAR.CESM2.1pctCO2.Omon.gn']

In [ ]:
preproc_dset_dict['CMIP.NCAR.CESM2.1pctCO2.r1i1p1f1.Omon.hfds.gn.gs://cmip6/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Omon/hfds/gn/v20190425/.nan.20190425']

In [ ]:
dset_dict['CMIP.MPI-M.MPI-ESM1-2-LR.1pctCO2.Omon.gn']

In [ ]:
preproc_dset_dict['CMIP.MPI-M.MPI-ESM1-2-LR.1pctCO2.r1i1p1f1.Omon.hfds.gn.gs://cmip6/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/1pctCO2/r1i1p1f1/Omon/hfds/gn/v20190710/.nan.20190710']

This modifies or adds various coordinates for consistency across models. You can see all of the included changes in the [`combined_preprocessing` documentation](https://cmip6-preprocessing.readthedocs.io/en/latest/_modules/cmip6_preprocessing/preprocessing.html#combined_preprocessing). In particular, the dimensions we will be using for averaging are now `x` and `y`. We can also use `replace_x_y_nominal_lat_lon` to change the values of `x` and `y` from cell indices to the actual lat/lon values.

### Weighted averages

In order to take averages of a variable over a region, we need the areas of the grid cells. We can use the provided `areacello` for this, and the process of loading this data is similar. We will also use the `match_metrics` function to add `areacello` for each Dataset we have been looking at.

In [ ]:
from cmip6_preprocessing.postprocessing import match_metrics

In [ ]:
cat_area = col.search(institution_id=['NCAR', 'MPI-M'], experiment_id='1pctCO2', variable_id='areacello', grid_label='gn')
cat_area.df

In [ ]:
dset_dict_area = cat_area.to_dataset_dict(zarr_kwargs={'consolidated': True}, aggregate=False, preprocess=combined_preprocessing)
list(dset_dict_area.keys())

Here is one of the `areacello` Datasets:

In [ ]:
dset_dict_area['CMIP.NCAR.CESM2.1pctCO2.r1i1p1f1.Ofx.areacello.gn.gs://cmip6/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Ofx/areacello/gn/v20190425/.nan.20190425']

Now we combine the two dictionaries: one with `hfds` and one with `areacello`.

In [ ]:
dset_dict_matched = match_metrics(preproc_dset_dict, dset_dict_area, ['areacello'])

This puts `areacello` under coordinates:

In [ ]:
dset_dict_matched['CMIP.MPI-M.MPI-ESM1-2-HR.1pctCO2.r1i1p1f1.Omon.hfds.gn.gs://cmip6/CMIP6/CMIP/MPI-M/MPI-ESM1-2-HR/1pctCO2/r1i1p1f1/Omon/hfds/gn/v20190710/.nan.20190710']

We can use this new dictionary to plot, for example, the weighted average `hfds` for one of the models over time.

In [ ]:
preproc_cesm2 = replace_x_y_nominal_lat_lon(dset_dict_matched['CMIP.NCAR.CESM2.1pctCO2.r1i1p1f1.Omon.hfds.gn.gs://cmip6/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Omon/hfds/gn/v20190425/.nan.20190425'])
preproc_cesm2

In [ ]:
cesm2_area = preproc_cesm2.areacello.fillna(0)

Let's also compare to the unweighted version:

In [ ]:
cesm2_mean_hfds = preproc_cesm2.hfds.weighted(cesm2_area).mean(['x', 'y'])
cesm2_mean_hfds_unweighted = preproc_cesm2.hfds.mean(['x', 'y'])

In [ ]:
cesm2_mean_hfds.plot()
cesm2_mean_hfds_unweighted.plot()

And with a nicer plot:

In [ ]:
# fig1 = plt.figure(1, dpi=130)
# ax = fig1.add_subplot(1, 1, 1)

# ax.plot(cesm2_mean_hfds.time, cesm2_mean_hfds, '--', alpha=0.3, color='b')
# ax.plot(cesm2_mean_hfds.time, cesm2_mean_hfds.rolling(time=12, center=True).mean(), color='b', label='Weighted (12MRM)')
# ax.plot(cesm2_mean_hfds_unweighted.time, cesm2_mean_hfds_unweighted, '--', alpha=0.2, color='g')
# ax.plot(cesm2_mean_hfds_unweighted.time, cesm2_mean_hfds_unweighted.rolling(time=12, center=True).mean(), color='g', label='Unweighted (12MRM)')

# plt.xlabel('Time (year)')
# plt.ylabel('hfds (W m$^{-2}$)')
# plt.title('Global average downward heat flux at sea surface (CESM2, 1pctCO2)')
# plt.grid()
# plt.legend()

### Unifying time across models

Note that CESM starts at year 1, while MPI-ESM starts at 1850. The starting year is arbitrary for the 1pctCO2 experiment (as well as others like abrupt-4xCO2), but if we try plotting a timeseries including both of these Datasets, they do not line up:

In [ ]:
cesm2 = dset_dict_matched['CMIP.NCAR.CESM2.1pctCO2.r1i1p1f1.Omon.hfds.gn.gs://cmip6/CMIP6/CMIP/NCAR/CESM2/1pctCO2/r1i1p1f1/Omon/hfds/gn/v20190425/.nan.20190425']
mpiesmlr = dset_dict_matched['CMIP.MPI-M.MPI-ESM1-2-LR.1pctCO2.r1i1p1f1.Omon.hfds.gn.gs://cmip6/CMIP6/CMIP/MPI-M/MPI-ESM1-2-LR/1pctCO2/r1i1p1f1/Omon/hfds/gn/v20190710/.nan.20190710']

In [ ]:
cesm2

In [ ]:
mpiesmlr

In [ ]:
cesm2.hfds.weighted(cesm2.areacello.fillna(0)).mean(dim=['x', 'y']).plot()
mpiesmlr.hfds.weighted(mpiesmlr.areacello.fillna(0)).mean(dim=['x', 'y']).plot()

To fix this, we can use `unify_time` to make the years line up.

In [ ]:
from cmip6_preprocessing.drift_removal import unify_time, replace_time

In [ ]:
unif_cesm2, unif_mpiesmlr = unify_time(cesm2, mpiesmlr)

In [ ]:
unif_cesm2

In [ ]:
unif_mpiesmlr

In [ ]:
unif_cesm2.hfds.weighted(cesm2.areacello.fillna(0)).mean(dim=['x', 'y']).plot()
unif_mpiesmlr.hfds.weighted(mpiesmlr.areacello.fillna(0)).mean(dim=['x', 'y']).plot()

## Summary
Add one final `---` marking the end of your body of content, and then conclude with a brief single paragraph summarizing at a high level the key pieces that were learned and how they tied to your objectives. Look to reiterate what the most important takeaways were.

### What's next?
Let Jupyter book tie this to the next (sequential) piece of content that people could move on to down below and in the sidebar. However, if this page uniquely enables your reader to tackle other nonsequential concepts throughout this book, or even external content, link to it here!

## Resources and references
Finally, be rigorous in your citations and references as necessary. Give credit where credit is due. Also, feel free to link to relevant external material, further reading, documentation, etc. Then you're done! Give yourself a quick review, a high five, and send us a pull request. A few final notes:
 - `Kernel > Restart Kernel and Run All Cells...` to confirm that your notebook will cleanly run from start to finish
 - `Kernel > Restart Kernel and Clear All Outputs...` before committing your notebook, our machines will do the heavy lifting
 - Take credit! Provide author contact information if you'd like; if so, consider adding information here at the bottom of your notebook
 - Give credit! Attribute appropriate authorship for referenced code, information, images, etc.
 - Only include what you're legally allowed: **no copyright infringement or plagiarism**
 
Thank you for your contribution!